In [ ]:
import pandas as pd

import numpy as np

# Introduction
The sinking of the Titanic is one of the most notirious shipwrecks in the history.In 1912 during her voyage,the Titanic sank after colliding with an iceberg.
Killing 1582 out of 2224 passengers and crew.

<font color ="blue">
Content:

[1.Load and Check](#1) 
    
[2.Variable Description](#2)
    
    
 *    [Univariate Variable Analysis](#3)
   
 *    [Categorical Variable Analysis](#4) 
    
 *    [Numerical Variable Analysis](#5)
    
    
 [3.Basic Data Analysis](#6)  
    
 [4.Outlier Data Detection](#7)   
    
 [5.Missing Value](#8)
     
  *    [Find Missing Value](#9)
    
  *    [Fill Missing Value](#10)
    
    
 [6.Visualization](#11)  
    
  *    [Correlation between SibSp-- Age -- Pclass -Parch -Fare- Survived](#12)
    
  *    [SibSp-Survived](#13)
    
  *    [Parch-Survived](#14)  
    
  *    [Fill Missing Age Feature](#15)  
    
    
  [7.Feature Engineering](#16)
    
       
   *    [Name-Title](#17)  
    
   *    [Family-Size](#18)
    
   *    [Embarked](#19)
    
   *    [Ticket](#20)
    
   *    [Pclass](#21)
    
   *    [Sex](#22) 
    
   *    [Drop Passenger ID and Cabin](#23) 
    
  
 [8.Modelling](#24) 
    
   *  [Train-Test-Split](#25) 
   
   *  [Simple Logistic Regression](#26)
      
   *  [Hyperparameter Tuning](#27)
   
   *  [Ensemble Modelling](#28) 
    
   *  [Prediction and Submission](#29) 

In [ ]:
import matplotlib.pyplot as plt

plt.style.use("seaborn-whitegrid")

import seaborn as sns

from collections import Counter

import warnings

warnings.filterwarnings("ignore")




In [ ]:
#plt.style.available(görselleştirmede kullanabilecek styllar)

<a id= "1"></a><br>
# Load and Check data

In [ ]:
train_df=pd.read_csv("/kaggle/input/titanic/train.csv")
test_df=pd.read_csv("/kaggle/input/titanic/test.csv")
test_PassengerId=test_df["PassengerId"]

In [ ]:
train_df.columns

In [ ]:
train_df.head()

In [ ]:
train_df.describe()

In [ ]:
train_df.info()

<a id= "2"></a><br>

# Variable Description
1. PassengerId:unique id number to each passenger
1. Survived :   passengere survived (1) or died (2)
1. Pclass   :   passenger seat class
1. Name     :   passenger name
1. Sex      :   passenger sex type
1. Age      :   passenger age
1. SibSp    :   siblings number
1. Parch    :   parents number
1. Ticket   :   ticket number
1. Fare     :   amount of money spent on ticket
1. Cabin    :   cabin category
1. Embarked :   port where passenger embarked


* float64(2):Age, Fare
* int64(5) :PassengerId,Survived,Pclass,sibsp,parch
* object(5):Nam****e,Sex,Cabin,Ticket,Embarked

<a id= "3"></a><br>
# Univarite Varaible Analysis

Catogorical Variables:Survived ,Pclass,Name,Sex,SibSp,Parch,Cabin,Embarked
Numerical Variables:Age,Fare,PassengerId	

<a id= "4"></a><br>
# Categorical Variable Analysis


In [ ]:
def bar_plot(variable):
    #get feature
    var=train_df[variable]
    
    #count number of categorical values
    var_value=var.value_counts()
    
    #visualization
    plt.figure(figsize=(4,10))
    plt.bar(var_value.index,var_value)
    plt.xticks(var_value.index,var_value.index.values)
    plt.ylabel("Frequency")
    plt.title("Variable")
    plt.show()
    print(" {}: \n {}".format(variable,var_value))
    

In [ ]:
category1=["Survived","Sex","Pclass","Embarked","SibSp","Parch"]
for c in category1:
    bar_plot(c)

<a id= "5"></a><br>
# Numerical Variable Analysis

In [ ]:
def plot_hist(variable):
    plt.xlabel(variable)
    plt.ylabel("Frequency")
    plt.title("{} distrubition with histogram".format(variable))
    plt.hist(train_df[variable])
    plt.show()
    

In [ ]:
Numeric_var=["Fare","Age","PassengerId"]

for i in Numeric_var:
    plot_hist(i)

<a id= "6"></a><br>
# Basic Data Analysis

* Pclass-Survived
* SibSp-Survived
* Patch-Survived
* Sex-Survived


In [ ]:
#Pclass-Survived involvement
train_df[["Pclass","Survived"]].groupby(["Pclass"],as_index=False).mean()

we can say that there is clear corelation between Pclass and Survived statistics

In [ ]:
#Sex-Survived involvement
train_df[["Sex","Survived"]].groupby(["Sex"],as_index=False).mean().sort_values(by="Survived",ascending=False)

In [ ]:
#SibSp-Survived involvement
train_df[["SibSp","Survived"]].groupby(["SibSp"],as_index=False).mean().sort_values(by="Survived",ascending=False)

In [ ]:
train_df[["Parch","Survived"]].groupby(["Parch"],as_index=False).mean().sort_values(by="Survived",ascending=False)

<a id= "7"></a><br>
# Outlier Data Detection

In [ ]:
def detect_outliers(train_df,features):
    outlier_indices = []
    
    for c in features:
        #1 quartile:
        Q1= np.percentile(train_df[c],25)
        
        #3 quartile:
        Q3= np.percentile(train_df[c],75)
        #IQR
        IQR= Q3 - Q1
        
        #Outlier Step
        outlier_step=IQR * 1.5
        
        #Detect Outlier and their indeces
        outlier_list_col=train_df[(train_df[c] < Q1- outlier_step) | (train_df[c] > Q3+ outlier_step)].index
        
        #store indices
        outlier_indices.extend(outlier_list_col)
        
    outlier_indices= Counter(outlier_indices)
    
    multiple_outliers= list(i for i, v in outlier_indices.items() if v >2)
    
    return multiple_outliers
          
    

In [ ]:
train_df.loc[detect_outliers(train_df,["Parch","Fare","Age","SibSp"])]
        

In [ ]:
#droping process

train_df=train_df.drop(detect_outliers(train_df,["Parch","Fare","Age","SibSp"]),axis=0).reset_index(drop=True)
train_df.head()

<a id="8"></a><br>
#  5.Missing Value
     
  *    Find Missing Value
  *    Fill Missing Value
  

In [ ]:
train_df_len=len(train_df)
train_df=pd.concat([train_df,test_df],axis=0).reset_index(drop=True)


In [ ]:
train_df.head()

<a id="9"></a><br>
Find Missing Value

In [ ]:
train_df.columns[train_df.isnull().any()]


In [ ]:
train_df.isnull().sum()

<a id="10"></a><br>
## Fill Missing Value


In [ ]:
train_df[train_df["Embarked"].isnull()]


In [ ]:
train_df.boxplot(column="Fare",by="Embarked"); # passengers who paid 80$ got on generally C point so we can fill the missing values on embarked through "C".

In [ ]:
train_df["Embarked"]=train_df["Embarked"].fillna("C")
train_df[train_df["Embarked"].isnull()]

In [ ]:
train_df[train_df["Fare"].isnull()]

In [ ]:
train_df["Fare"]=train_df["Fare"].fillna(np.mean(train_df[train_df["Pclass"]==3]["Fare"]))

In [ ]:
train_df[train_df["Fare"].isnull()] #we filled the missing values with Fare average

<a id="11"></a><br>
## Visualization
<a id="12"></a><br>
Correlation between SibSp-- Age -- Pclass -Parch -Fare- Survived

In [ ]:


import seaborn as sns
list1=["SibSp","Age","Pclass","Parch","Fare","Survived"]

sns.heatmap(train_df[list1].corr(),annot=True,fmt=".2f") #annot=boxların içindeki değerleri ifade eder. #fmt=değerlerin vigülden sonraki basamak sayısını ifade eder
plt.show()




<a id="13"></a><br>
SibSp-Survived

In [ ]:
sns.factorplot(x="SibSp",y="Survived",data=train_df,kind="bar")
plt.ylabel("survived probability")
plt.xlabel("number of siblings")
plt.title("the correlation between survived probability and number of siblings")
plt.show()

Parch--Survived
<a id="14"></a><br>

In [ ]:
sns.factorplot(x="Parch",y="Survived",data=train_df,kind="bar")
plt.ylabel("survived probability")
plt.xlabel("large of parch")
plt.title("the correlation between survived probability and number of siblings")
plt.show()

<a id="15"></a><br>
Fill Missing :Age Feature

In [ ]:
train_df[train_df["Age"].isnull()]

In [ ]:
sns.boxplot(x="Sex",y="Age",hue="Pclass",data=train_df)
plt.show()

In [ ]:
sns.boxplot(x="Parch",y="Age",data=train_df)
plt.show()

sns.boxplot(x="SibSp",y="Age",data=train_df)
plt.show()

In [ ]:
train_df["Sex"]=[1 if i =="male" else 0 for i in train_df["Sex"] ]

In [ ]:
sns.heatmap(train_df[["Sex","SibSp","Pclass","Parch","Age"]].corr(),annot=True)

In [ ]:
index_nan_age=list(train_df["Age"][train_df["Age"].isnull()].index)
index_nan_age

In [ ]:
for i in index_nan_age:
    age_pred=train_df["Age"][train_df["SibSp"]==train_df.iloc[i]["SibSp"]].median()
    age_med=train_df["Age"].median()
    
    if not np.isnan(age_pred):
        train_df["Age"].iloc[i]=age_pred
    else:
        train_df["Age"].iloc[i]=age_med

In [ ]:
train_df[train_df["Age"].isnull()]

<a id="16"></a><br>
# Feature Engineering
 


Name---Title

In [ ]:
name=train_df["Name"]
train_df["Title"]=[i.split(".")[0].split(",")[-1].strip() for i in name]  #öncelikle . ya göre satırı ikiye ayırdık ve birinci kısmı seçtik(Title ın olduğu kısım).Daha sonra ise , ile ayırarak son kısmı seçtik.

In [ ]:
train_df["Title"]

In [ ]:
sns.countplot(x="Title",data=train_df)
plt.xticks(rotation=50)
plt.show

In [ ]:
#convert to categorical
train_df["Title"]=train_df["Title"].replace(["Lady","Capt","Col","the Countess","Jonkheer","Dona","Major","Rec","Dr","Don","Sir"],"other")


In [ ]:
train_df["Title"]=[0 if i =="Master" else 1 if i == "Miss" or i=="Ms" or i=="Mlle" or i=="Mrs" else 2 if i =="Mr" else 3 for i in train_df["Title"]]

In [ ]:
sns.countplot(x="Title",data=train_df)
plt.xticks(rotation=50)
plt.show

In [ ]:
g=sns.barplot(x="Title",y="Survived",data=train_df)
g.set_xticklabels(["Master","Mrs","Mr","Other"])  # x değerleri köşeli parantez içinde olmalı  
plt.show()

In [ ]:
train_df.drop(labels=["Name"],inplace=True,axis=1)  #axis=1 unutulmamalı 

In [ ]:
train_df=pd.get_dummies(train_df,columns=["Title"])#get_dummies in işlevi=nümeric değerlere dönüşen Title columnunu 1 ve 0 olarak sınıflandırmak


In [ ]:
train_df

<a id= "18"></a><br>
# Family---Size

In [ ]:
train_df["Fsize"]=train_df["Parch"]+train_df["SibSp"]+1 # +1 in sebebi ,tek kişiyi de bir kişilik aile olarak düşünebiliriz
train_df

In [ ]:
g=sns.factorplot(x="Fsize",y="Survived",data=train_df,kind="bar")
g.set_ylabels("Survival pprobability")
g.set_xlabels("Family Size")
plt.show()

In [ ]:
train_df["Family_size"]=[1 if i< 5 else 0 for i in train_df["Fsize"]]

In [ ]:
train_df.head(10)

In [ ]:
sns.countplot(x="Family_size",data=train_df); #countplotta y değeri kullanmamalısın

**small families have more chance to survive than large families according to data

In [ ]:
train_df=pd.get_dummies(train_df,columns=["Family_size"])

In [ ]:
train_df.head(10)

<a id= "19"></a><br>
## Embarked



In [ ]:
g=sns.countplot(x="Embarked",data=train_df)

plt.show()

In [ ]:
train_df["Embarked"]=[0 if i=="S" else 1 if i =="C" else 2  for i in train_df["Embarked"]]
train_df.head(10)

In [ ]:
train_df=pd.get_dummies(train_df,columns=["Embarked"])
train_df

<a id= "20"></a><br>
# Ticket

In [ ]:
train_df["Ticket"]

In [ ]:
tickets=[]
for i in list(train_df.Ticket):
    if not i.isdigit():
        tickets.append(i.replace(".","").replace("/","").strip().split()[0])
    else:
             tickets.append("x")   

In [ ]:
#example for display
a="SOTON/O.Q. 3101262"

a.replace(".","").replace("/","").strip().split()[0]

In [ ]:

train_df["Ticket"]=tickets
train_df["Ticket"].head(10)

In [ ]:
train_df=pd.get_dummies(train_df,columns=["Ticket"],prefix="T")

In [ ]:
train_df.head(20)

<a id= "21"></a><br>
# Pclass

In [ ]:
sns.countplot(x="Pclass",data=train_df);

In [ ]:
train_df["Pclass"]=train_df["Pclass"].astype("category")
train_df=pd.get_dummies(train_df,columns=["Pclass"])

In [ ]:
train_df

<a id= "22"></a><br>
# Sex

In [ ]:
train_df["Sex"]=train_df["Sex"].astype("category")
train_df=pd.get_dummies(train_df,columns=["Sex"])

In [ ]:
train_df.head(10)

<a id= "23"></a><br>
    
# Dropping Passenger ID and Cabin

In [ ]:
train_df.drop(labels=["PassengerId","Cabin"],axis=1,inplace=True)
train_df

<a id= "24"></a><br>
# Modelling

In [ ]:
from sklearn.model_selection import train_test_split ,GridSearchCV,StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier,VotingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

<a id= "25"></a><br>

# Train-Test-Split

In [ ]:
train_df_len

In [ ]:
test=train_df[train_df_len:]
test.drop(labels=["Survived"],axis=1,inplace=True)

In [ ]:
test.head()

In [ ]:
train=train_df[:train_df_len]
x_train=train.drop(labels="Survived",axis=1)
y_train=train["Survived"]
x_train,x_test,y_train,y_test=train_test_split(x_train,y_train,test_size=0.3,random_state=42)

In [ ]:
print("x_test=",len(x_test))
print("x_train=",len(x_train))
print("y_test=",len(y_test))
print("y_train=",len(y_train))

<a id= "26"></a><br>

# Simple Logistic Regression

In [ ]:
logreg=LogisticRegression()
logreg_model=logreg.fit(x_train,y_train)


In [ ]:
logreg_pred=logreg_model.predict(x_test)


In [ ]:
#accuracy_score
accuracy_score_train=logreg.score(x_train,y_train)
accuracy_score_train

In [ ]:
accuracy_score_test=logreg.score(x_test,y_test)
accuracy_score_test

<a id= "27"></a><br>
## Hyperparameter Tuning
*   Decision Tree
*   SVM
*   Random Forest
*   KNN
*   Logistic Regression

In [ ]:
classifier=[DecisionTreeClassifier(random_state=42),
            SVC(random_state=42),
            RandomForestClassifier(random_state=42),
            LogisticRegression(random_state=42),
            KNeighborsClassifier()]
dt_params={"min_samples_split":range(10,500,20),"max_depth":range(10,20,2)}

svc_params= {"kernel":["rbf"],
             "gamma":[0.001,0.01,0.1],
             "C":[10,30,50,100,200]}

rf_params={"min_samples_split":[1,3,10],
          "max_features":[1,3,10],
          "bootstrap":[False],
          "n_estimators":[100,300],
          "criterion":["gini"]}

logreg_params={"C":np.logspace(-3,3,7),
              "penalty":["l1","l2"]}

knn_params={"n_neighbors":np.linspace(1,19,10,dtype=int).tolist(),
           "weights":["uniform","distance"],
           "metric":["euclidean","manhattan"]}

classifier_params=[dt_params,svc_params,rf_params,logreg_params,knn_params]

In [ ]:
cv_result=[]
best_estimators=[]
for i in range(len(classifier)):
    clf=GridSearchCV(classifier[i],param_grid=classifier_params[i],cv=StratifiedKFold(n_splits=10))
    
    clf.fit(x_train,y_train)
    
    cv_result.append(clf.best_score_)
    best_estimators.append(clf.best_estimator_)
    print("best score for {}".format(classifier[i]),cv_result[i])


In [ ]:
print("best estimators for {}".format(classifier[i]),best_estimators)

In [ ]:
cv_result=pd.DataFrame({"ML Models":["DecisionTreeClassifier","SVC","RandomForestClassifier","LogisticRegression","KNeighborsClassifier"],"Cross Validation Means":cv_result})
cv_result






In [ ]:
sns.barplot(x="ML Models",y="Cross Validation Means",data=cv_result)
plt.xticks(rotation=25)

<a id= "28"></a><br>
# Ensemble Modelling

In [ ]:
votingC=VotingClassifier(estimators=[("dt",best_estimators[0]),("rfc",best_estimators[2]),("lr",best_estimators[3])], voting = "soft",n_jobs=-1)
votingC=votingC.fit(x_train,y_train)
votingC_pred=votingC.predict(x_test)

In [ ]:
print("accuracy score for test:",accuracy_score(votingC_pred,y_test))

<a id= "29"></a><br>
## Prediction and Submission

In [ ]:
test_survived=pd.Series(votingC_pred,name="Survived").astype(int)
results=pd.concat([test_PassengerId,test_survived],axis=1)
results.to_csv("titanic.csv",sep=",",index=False)